We shall outline the process to work with clinical trials data to parse the XML file. This can be further extended to multiple use cases for other data as well

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#Getting the imports done
import numpy as np 
import pandas as pd 
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import xml.etree.ElementTree as ET

The XML file can be viewed as 'organized as a tree'. Fortunately, an inspection of a single XML file in several cases can help us know the root node, leaves, etc. which can be leveraged to extract key information. We have followed the exact same process here. 

In [0]:
#Utility function to parse our XML file
def xml_csv_convert(xml_file):
  tree = ET.parse(xml_file)
  root = tree.getroot()
  #The key columns that we want
  title_text = ""
  detailed_description = ""
  summary = ""
  fac_name = ""
  fac_address = ""
  status = ""
  source = ""

  for desc in root.iter("detailed_description"):
    for txt in desc.findall('textblock'):
      detailed_desc = txt.text
      detailed_desc = detailed_desc.replace('\n','')
      detailed_desc = detailed_desc.replace(';',',')
      detailed_desc=re.sub(' +',' ',detailed_desc)
      detailed_description = detailed_desc
  for summ in root.iter("brief_summary"):
    for txt in summ.findall('textblock'):
      summary_text = txt.text
      summary_text = summary_text.replace('\n','')
      summary_text = summary_text.replace(';',',')
      summary_text=re.sub(' +',' ',summary_text)
      summary = summary_text

  for title in root.iter("official_title"):
    title_text = title.text
  for loc in root.iter("facility"):
    for t in loc.findall('name'):
      fac_name = t.text
    for ad in loc.findall('address'):
      fac_address = ad.find('city').text+", "+ad.find('country').text
  for stat in root.iter("overall_status"):
    status = stat.text
  for sr in root.iter("source"):
    source = sr.text
  #Creating a string in the correct format to write to the CSV. Note that we are using a semicolon delimiter  
  total_text = "\"" +title_text+"\"" +";"+"\""+summary +"\""+ ";" + "\"" + detailed_description + "\"" + ";" + "\"" + fac_name + "\"" + ";" + "\"" + fac_address + "\"" + ";" + "\"" + status + "\"" + ";" + "\"" + source + "\""
  return total_text




The XML files are separate for each clinical trial so we need to iterate over each of the XML files

In [0]:

import csv
subdir = '/content/gdrive/My Drive/search_result'
with open('clinical_data.csv','w',encoding='utf-8') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = ["title", "summary", "detailed_description", "facility_name", "facility_address", "status", "source"], delimiter = ";")
  writer.writeheader()
  for root,dir,files in os.walk('/content/gdrive/My Drive/search_result'):
    for name in files:
      print (xml_csv_convert(os.path.join(subdir,name)))
      csvfile.write(xml_csv_convert(os.path.join(subdir,name)))
      csvfile.write('\n')

  